# Let's do some biomechanics !!

Now, we are working with a arm model, actuated by muscles and pure joint torques (to compensate for model weakness). 
The goal is, starting from an initial position, to perform a reaching  task with this arm, by superimposing two markers. 
The first one (called 'target' in the model), is fixed in the scene.
The second one (called 'COM_hand' in the model) is fixed on the arm, in the *ulna* coordinate system.

The ocp to solve is as follow:
- Movement of $3s$ divided into $50$ shooting points
- The state variables are the generalized position (q) and velocities (qdot)
- The control variables are the muscle activations (a) 
- Cost functions: minimize the control effort at all time and minimize the squared Euclidean distance between the two markers at the final node
- Initial guess is arbitrarily set to $0.5$ for muscle activations and to $0$ for all other variables

In [ ]:
# So first, let's import all the required classes 
import biorbd_casadi as biorbd
from bioptim import *

In [ ]:
# Let's define the OCP
final_time = 0.5
n_shooting = 30

# Load the model and define some aliases
model = biorbd.Model("models/arm26.bioMod")
nq = model.nbQ()
ndq = model.nbQdot()
ntau = model.nbGeneralizedTorque()
nmus = model.nbMuscleTotal()

# Add objective functions
objective_functions = ...
objective_functions.add(...) # min torque, weight = 10
objective_functions.add(...) # min muscle control, weight = 1
objective_functions.add(..., first_marker=..., second_marker=..., ...) # superimpose markers, weight = 1000

# Dynamics
dynamics = ...
dynamics.add(...) # muscle act. and torque driven

# Path constraint
x_bounds = ...
x_bounds.add(bounds=QAndQDotBounds(model))
... # force initial state at (0.07, 1.4, 0, 0)

# Initial guess on state
x_init = ...
x_init.add([1.57] * nq + [0] * ndq)

# Define control path constraint
muscle_min, muscle_max, muscle_init = 0, 1, 0.5
tau_min, tau_max, tau_init = -5, 5, 0
u_bounds = BoundsList()
u_bounds.add(
    ..., # min bound
    ...,# max bound
)

# Initial guess on control
u_init = InitialGuessList()
u_init.add([tau_init] * ntau + [muscle_init] * nmus)
# ------------- #

ocp = OptimalControlProgram(...)


In [ ]:
sol = ocp.solve()

In [ ]:
sol.graphs(automatically_organize=False)

In [ ]:
# sol.animate()
# This would work locally on your computer. Unfortunately, it is nor currently working with remote notebooks...
# Call one of the workshop organizers, so you can see the motion animation.

In [ ]:
sol.print()

Let us compute the distance between the two markers at the end of the motion, based on the information printed in the console above. Keep in mind that SUPERIMPOSE_MARKERS is a quadratic cost.


In [ ]:
import numpy as np
# Prepare the casadi function to call
markers = biorbd.to_casadi_func("mark1", model.markers, ocp.nlp[0].states["q"].mx)

# Extract results
q = ...
target_idx = biorbd.marker_index(model, "target")
hand_idx = biorbd.marker_index(model, "COM_hand")

dist = np.sqrt(np.sum((markers(q)[:, target_idx] - markers(q)[:, hand_idx])**2))
print(f"At the final node, the distance between the 2 markers is {round(dist, 3) * 1000}m")


You find it too large ? Try to solve the problem again by increasing the weight of the objective on the markers matching !

In [ ]:
...